# Field visualisation using `k3d`

**Note:** If you experience any problems in plotting with `k3d`, please make sure you run the Jupyter notebook in Google Chrome.

There are several ways how a field can be visualised, using:

- `matplotlib`
-`k3d`
- `holoviews`
- vtk-based libraries, e.g. `pyvista`


`k3d` provides three-dimensional interactive plots of fields inside Jupyter notebook.

Let us say we have a sample, which is an ellipsoid

$$\frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2} <= 1$$

with $a=5\,\text{nm}$, $b=3\,\text{nm}$, and $c=2\,\text{nm}$. The space is discretised into cells with dimensions $(0.5\,\text{nm}, 0.5\,\text{nm}, 0.5\,\text{nm})$. The value of the field at $(x, y, z)$ point is $(-cy, cx, cz)$, with $c=10^{9}$. The norm of the field inside the cylinder is $10^{6}$.

Let us first build that field.

In [1]:
import discretisedfield as df

a, b, c = 5e-9, 3e-9, 2e-9
cell = (0.5e-9, 0.5e-9, 0.5e-9)

mesh = df.Mesh(p1=(-a, -b, -c), p2=(a, b, c), cell=cell)


def norm_fun(pos):
    x, y, z = pos
    if (x / a) ** 2 + (y / b) ** 2 + (z / c) ** 2 <= 1:
        return 1e6
    else:
        return 0


def value_fun(pos):
    x, y, z = pos
    c = 1e9
    return (-c * y, c * x, c * z)


field = df.Field(mesh, dim=3, value=value_fun, norm=norm_fun)

The most basic plot we can show is the plot of all the cells where the value is non-zero. This can be useful, to inspect the domain created, by plotting the norm.

In [2]:
field.norm.k3d.nonzero()

Output()

The plot is interactive, so it can be manipulated using a mouse. To change the color of voxels, we can pass the new color via `color` argument.

In [3]:
field.norm.k3d.nonzero(color=0x27AE60)

Output()

Next, we can plot a scalar field. For plotting a scalar field, we are using `discretisedfield.Field.k3d.scalar()` method.

In [4]:
field.k3d.scalar()

ValueError: Cannot plot dim=3 field.

An exception was raised because we attempted to plot a vector field using voxels. Therefore, we first need to extract a component of the field. Let us plot the $x$ component.

In [5]:
field.x.k3d.scalar()

Output()

However, we can see that the points which we consider to be outside the sample are also plotted. This is because, `discretisedfield.Field.k3d.scalar()` method cannot determine the points where norm is zero from the passed scalar field. Therefore, we need to pass the `norm_field.`

In [6]:
field.x.k3d.scalar(filter_field=field.norm, multiplier=1e-6)

Output()

By cascading operations, we can similarly plot the slice of the ellipsoid at $z=0$.

In [7]:
field.x.plane("z").k3d.scalar(filter_field=field.plane("z").norm)

Output()

To further modify the plot, keyword arguments for `k3d.factory.voxels()` function are accepted. Pleaase refer to its [documentation](https://k3d-jupyter.org/k3d.html#k3d.factory.voxels)

Next, we can plot the vector field itself.

In [8]:
field.k3d.vector()

Output()

By default, points at the discretisation cell centres are plotted together with vectors to help understand the structure of the field. However, they can be deactivated by passing `points=False`.

In [9]:
field.k3d.vector(points=False)

Output()

It is difficult to understand the vector field from this plot. By cascading, we can plot its slice at $x=0$.

In [10]:
field.plane(x=0).k3d.vector()

Output()

To improve the understanding of the plot, we can now colour the vectors plotted. For that, we need to pass a scalar field, according to which the vectors will be coloured.

In [11]:
field.plane(x=0).k3d.vector(color_field=field.x)

Output()

To further modify the plot, keyword arguments for `k3d.factory.vectors()` function are accepted. Plese refer to its [documentation](https://k3d-jupyter.org/k3d.html#k3d.factory.vectors).

### Multiple visualisation on the same plot

Sometimes, it is necessary to show, for example, multiple planes of the sample on the same plot. This can be done by exposing the `k3d.plot` and passing it to different plotting methods. For instance.

In [12]:
import k3d

plot = k3d.plot()
field.plane(x=-3e-9).k3d.vector(plot=plot, color_field=field.z)
field.plane(x=0).k3d.vector(plot=plot, color_field=field.z, cmap="hsv")
field.plane(x=3e-9).k3d.vector(plot=plot, color_field=field.z)
plot.display()

Output()

## `k3d` interactive plots

In [13]:
field.plane(z=0).z.k3d.scalar(filter_field=field.norm)

Output()

In [14]:
import k3d

plot1 = k3d.plot()


@df.interact(z=field.mesh.slider("z", continuous_update=True))
def myplot(z):
    field.z.plane(z=z).k3d.scalar(
        filter_field=field.norm, interactive_field=field, plot=plot1
    )


plot1.display()

interactive(children=(SelectionSlider(description='z (nm)', index=4, options=((-1.75, -1.7500000000000002e-09)…

Output()

In [15]:
plot = k3d.plot()


@df.interact(
    z=field.mesh.slider("z", continuous_update=True), axis=field.mesh.axis_selector()
)
def myplot(z, axis):
    getattr(field, axis).plane(z=z).k3d.scalar(
        filter_field=field.norm, interactive_field=field, plot=plot
    )


plot.display()

interactive(children=(SelectionSlider(description='z (nm)', index=4, options=((-1.75, -1.7500000000000002e-09)…

Output()

In [16]:
plot = k3d.plot()


@df.interact(
    y=field.mesh.slider("y", continuous_update=True), axis=field.mesh.axis_selector()
)
def myplot(y, axis):
    field.plane(y=y).k3d.vector(
        color_field=getattr(field, axis), interactive_field=field, plot=plot
    )


plot.display()

interactive(children=(SelectionSlider(description='y (nm)', index=6, options=((-2.75, -2.75e-09), (-2.25, -2.2…

Output()